In [ ]:
import rubin_nights.dayobs_utils as rn_dayobs

day_obs = rn_dayobs.day_obs_str_to_int(rn_dayobs.today_day_obs())
#day_obs = 20251101
sim_nights = 1

In [ ]:
ts_config_scheduler_root = "/sdf/data/rubin/shared/scheduler/times_square/ts_config_scheduler"
!/bin/rm -rf $ts_config_scheduler_root